# Introduction

Hello people,welcome to my kernel. In this kernel I am going to use international airline passangers dataset. It is a famous dataset for LSTM exercise. Today, I will predict how many passangers came in a day. In order to do this I am going to create a LSTM model using keras library.

Before the starting, let's take a look at our content

# Notebook Content
1. Loading and Visualizing Data
1. Data Preprocessing
1. Creating Model
1. Predicting and Visualizing
1. Conclusion


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import seaborn as sns
sns.set_style('darkgrid')
from matplotlib import pyplot as plt


import math
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Loading And Visualizing Data
In this section I am going to import the data, then I am going to take a look.

In [ ]:
data = pd.read_csv('/kaggle/input/international-airline-passengers/international-airline-passengers.csv',skipfooter=5)
passanger_values = data.iloc[:,1].values

In [ ]:
data.head()

* There is a problem in the 2nd column's name. You can clean but in this kernel I won't do.

In [ ]:
fig,ax = plt.subplots(figsize=(10,7))
plt.plot(passanger_values,color="green")
plt.xlabel("Time")
plt.ylabel("Passanger Count (Thousand)")
plt.show()

* The data is seasonal.
* The number of passengers has increased over the years

# Data Preprocessing

In this section I am going to preapre the data in order to use in our future model. I am going to follow these steps:
* Reshaping
* Changing Type
* Scaling
* Train Test Splitting
* Creating X and Y 


In [ ]:
passanger_values = passanger_values.reshape(-1,1) # Converting scalar into vector
passanger_values = passanger_values.astype("float32") # Changing Type 
passanger_values.shape

In [ ]:
scaler = MinMaxScaler(feature_range=(0,1)) # Scaling
passanger_values = scaler.fit_transform(passanger_values)
passanger_values[:5]

In [ ]:
train = []
test = []

train_len = 71
test_len = 71

train = passanger_values[:71]
test = passanger_values[71:]

print("The length of train",len(train))
print("The length of test",len(test))

In [ ]:
time_stemp = 10
dataX = []
dataY = []
for i in range(len(train)-time_stemp-1):
    a = train[i:(i+time_stemp), 0]
    dataX.append(a)
    dataY.append(train[i + time_stemp, 0])
trainX = np.array(dataX)
trainY = np.array(dataY)  
print("The shape of trainX",trainX.shape)
print("The shape of trainY",trainY.shape)

In [ ]:
time_step = 10
dataX,dataY = [],[]
for i in range(len(train)-time_step-1):
    a = test[i:(i+time_step), 0]
    dataX.append(a)
    dataY.append(test[i + time_step, 0])
testX = np.array(dataX)
testY = np.array(dataY)
print("The shape of testX",testX.shape)
print("The shape of testY",testY.shape)

In [ ]:
testX = testX.reshape((60,1,10))
trainX = trainX.reshape((60,1,10))
print("The shape of testX",testX.shape)
print("The shape of trainX",trainX.shape)

# Creating Model
Data is ready, in this section we will create a simple LSTM model using keras library. At least for this kernel, I'll create a very simple model because our data is little.

In [ ]:
from keras.layers import Dense,Dropout,LSTM
from keras.models import Sequential

regressor = Sequential()
regressor.add(LSTM(10,input_shape = (1,10)))
regressor.add(Dense(1))
regressor.compile(optimizer = "adam",loss="mean_squared_error")

In [ ]:
regressor.fit(trainX,trainY,epochs=50)

# Predicting and Visualizing
Our model is fitted and ready. In this section I am going to apply predictions and after that I am going to evaulate them.

In [ ]:
#Predicting
trainPred =regressor.predict(trainX) 
testPred = regressor.predict(testX)

#Taking Inverse
trainPred = scaler.inverse_transform(trainPred)
testPred = scaler.inverse_transform(testPred)

trainY = scaler.inverse_transform([trainY])
testY = scaler.inverse_transform([testY])



In [ ]:
#Computing RMSE (Root Mean Squared Error)
trainScore = math.sqrt(mean_squared_error(trainY[0], trainPred[:,0]))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = math.sqrt(mean_squared_error(testY[0], testPred[:,0]))
print('Test Score: %.2f RMSE' % (testScore))

In [ ]:
# shifting train
trainPredictPlot = np.empty_like(passanger_values)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[time_step:len(trainPred)+time_step, :] = trainPred


# shifting test predictions for plotting
testPredictPlot = np.empty_like(passanger_values)
testPredictPlot[:, :] = np.nan
testPredictPlot[len(trainPred)+(time_step*2)+1:len(passanger_values)-1, :] = testPred


# plot baseline and predictions
fig,ax = plt.subplots(figsize=(10,7))
plt.plot(scaler.inverse_transform(passanger_values))
plt.plot(trainPredictPlot)
plt.plot(testPredictPlot)
plt.show()

* As we can see our result is great for a simple model.


# Conclusion
Thanks for your attention, if you have questions in your mind, you can ask me in the comment section. I am waiting for your questions, comments and upvotes.

Have a great day!